# Projet 2 : Analyser des données de systèmes éducatifs
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours DataScientist.  

L'objectif principal est de réaliser une analyse exploratoire de données pour une compagnie fictive EdTech company appelée *Academy*. Cette compagnie, spécialisée dans la formation en ligne à destination des lycées et des étudiants en étude supérieure, cherche à se développer à l'international. Elle nous demande de lui trouver les meilleurs pays cibles, ne disposant pas encore de données pertinantes en interne elle propose d'utiliser les données de la banque mondiale.
https://datacatalog.worldbank.org/dataset/education-statistics

Notre première mission, sera de déterminer si ces données seront suffisantes pour mener à bien notre étude.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import analysis.dataload as ld
import analysis.datavisu as vs
import analysis.datastats as st
import seaborn as sns

ModuleNotFoundError: No module named 'analysis.datastats'

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Récupération-des-données" data-toc-modified-id="Récupération-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Récupération des données</a></span><ul class="toc-item"><li><span><a href="#Télécharger-les-données" data-toc-modified-id="Télécharger-les-données-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Télécharger les données</a></span></li><li><span><a href="#Découvrir-les-données" data-toc-modified-id="Découvrir-les-données-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Découvrir les données</a></span></li><li><span><a href="#Sélection-des-variables" data-toc-modified-id="Sélection-des-variables-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Sélection des variables</a></span></li></ul></li><li><span><a href="#Explorer-les-données" data-toc-modified-id="Explorer-les-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Explorer les données</a></span><ul class="toc-item"><li><span><a href="#Examiner-la-stucture-des-données" data-toc-modified-id="Examiner-la-stucture-des-données-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Examiner la stucture des données</a></span></li><li><span><a href="#Visualisations-des-données" data-toc-modified-id="Visualisations-des-données-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Visualisations des données</a></span></li><li><span><a href="#Recherche-de-corrélations-entre-les-variables-quantitatives" data-toc-modified-id="Recherche-de-corrélations-entre-les-variables-quantitatives-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Recherche de corrélations entre les variables quantitatives</a></span></li><li><span><a href="#Recherche-des-corrélations-avec-les-variables-qualitatives" data-toc-modified-id="Recherche-des-corrélations-avec-les-variables-qualitatives-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Recherche des corrélations avec les variables qualitatives</a></span></li></ul></li><li><span><a href="#Préparer-les-données" data-toc-modified-id="Préparer-les-données-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Préparer les données</a></span><ul class="toc-item"><li><span><a href="#Nettoyer-les-données" data-toc-modified-id="Nettoyer-les-données-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Nettoyer les données</a></span></li><li><span><a href="#Imputer-les-données-manquantes" data-toc-modified-id="Imputer-les-données-manquantes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Imputer les données manquantes</a></span></li></ul></li><li><span><a href="#Création-d'un-modèle-permettant-de-trier-les-pays-en-fonction-de-leur-pertinance" data-toc-modified-id="Création-d'un-modèle-permettant-de-trier-les-pays-en-fonction-de-leur-pertinance-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Création d'un modèle permettant de trier les pays en fonction de leur pertinance</a></span><ul class="toc-item"><li><span><a href="#Création-d'un-score" data-toc-modified-id="Création-d'un-score-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Création d'un score</a></span></li><li><span><a href="#Visualisation-des-5-pays-les-plus-pertinants" data-toc-modified-id="Visualisation-des-5-pays-les-plus-pertinants-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Visualisation des 5 pays les plus pertinants</a></span></li></ul></li></ul></div>

## Récupération des données

### Télécharger les données

On commence par télécharger les données du site de la banque mondiale. Pour cela nous utilisons une fonction spécifiquement créée dans ce but.

In [ ]:
url = 'http://databank.worldbank.org/data/download/Edstats_csv.zip'
list_csv = ld.download_data(url)

6 fichiers en format csv ont été téléchargés. Les deux premiers donnent des infos sur les attributs disponibles, le troisième informe sur la source des données, le quatrième est celui qui contient toutes les données et qui va donc être à la base de notre analyse. Les deux derniers fournissent des données supplémentaires sur les pays. Pour notre analyse nous utiliserons les deux fichiers suivants :
- EdStatsData.csv
- EdStatsCountry.csv

Pour manipuler les données, nous utiliserons la librairies pandas de Python.

In [2]:
raw_data = ld.load_data(list_csv[2])
raw_country = ld.load_data(list_csv[1])

NameError: name 'list_csv' is not defined

### Découvrir les données

Commençons par regarder ce que contiennent ces données, pour cela nous utiliserons la méthode `info()` de pandas qui permet d'avoir des infos génériques du type : nombre d'individus, nom des variables ou encore le nombre d'entrée non nulles pour chacune d'entre elles.

In [3]:
raw_data.info()

NameError: name 'raw_data' is not defined

On note plusieurs choses intéressantes. Premièrement sur les 70 variables (colonnes) disponibles, la plupart sont des années. Ces années s'échelonnent entre 1970 et 2100, avec un pas de 1 de 1970 à 2017 et un pas de 5 post 2020. Les années de 2020 à 2100 semblent être des projections. Ne sachant pas exactement sur quelles hypothèses elles ont été faites, il est préférable de ne pas les inclures dans notre analyse.

In [4]:
df = raw_data.iloc[:, :-18].copy()

NameError: name 'raw_data' is not defined

Les lignes (au nombre de 886'930) semblent correspondre à un attribut donné (colonne *Indicator Name*) pour un pays donné. Regardons combien de pays et d'attributs différents sont disponibles.

In [5]:
# display the number of countries
n_countries = df['Country Code'].unique().size
print("There are {} different countries.".format(n_countries))

# display the number of indicators
n_ind = df['Indicator Code'].unique().size
print("There are {} different indicators.".format(n_ind))

NameError: name 'df' is not defined

La bonne nouvelle c'est que les données sont exhaustives, par contre 3665 attributs c'est beaucoup trop, il va donc falloir sélectionner les plus pertinents. De plus, la configuration actuelle des données n'est pas très pratique. En effet, on aimerait avoir non pas les années en colonnes mais les attributs sélectionnés et en ligne les pays. Les valeurs utilisées pour remplir cette table seront celles de l'année la plus récente si suffisamment de données sont disponibles pour cette dernière, dans le cas contraire nous prendrons l'année précédente.

### Sélection des variables

Comme dis plus haut il va falloir sélectonner les attributs que l'on va utiliser. Pour se faire, revenons à l'objectif de notre mission. Il s'agit d'aider une compagnie spécialisée dans la formation en ligne pour les lycéens et les étudiants. On va donc viser **une population jeune (15-25 ans) qui a accés à internet**. De plus, pour pouvoir différencier les pays il peut être intéressant de connaître le PIB/habitants ainsi qu'une mesure des inégalités de chaque pays. Le premier nous permettra d'avoir une vague idée de la capacité financière de la population, quant à la deuxième elle nous permettra de savoir si le diplôme a un réel impact sur la carière et les revenus futurs. Ce dernier point peut paraître étrange, mais une étude de la banque mondiale de **XX**, repris pour le projet 7 du parcours de Data Analyst d'Open Classrooms permet de constater que, plus le niveau d'inégalité est élevé, plus ce sont les revenus des parents et non le diplôme qui vont déterminer les revenus futurs de l'enfant. Ainsi, la volonté de réussir sont parcours scolaire sera beaucoup moins présent dans les pays inégalitaires que les pays égalitaires.

Parmi les milliers d'indicateurs disponibles dans les données de la banque mondiale, les suivants se rapprochent des attributs décris ci-dessus :
- GDP per Capita, PPP
- Gini coeffient of average years of schooling
- Internet access
- Population, 15-24, total

Nous ajouterons aussi les données de population totale qui permettrons de calculer la part des jeunes (15-24 ans) sur la population totale.

Pour finir, nous ajouterons à partir du jeu de données sur les pays les variables :
- Region
- Income group

La première pour voir si on observe de grosses variations d'une région à l'autre, mais aussi au sein d'une même région. La deuxième pour vérifier s'il ne serait pas plus pertinent de travailler par classe de revenus plutôt que par revenu directement.

Nous sélectionnerons ces attributs pour les trois dernières années disponibles (2015-2017) au cas ou trop peu de données seraient disponibles pour 2017 et 2016.

In [6]:
# get the first four features from the mains dataset
columns = df.columns.to_list()
columns = columns[: 4] + columns[-3:]
list_feat = ['NY.GDP.PCAP.PP.CD',
             'HH.DHS.YRS.15UP.GIN',
             'HH.MICS.YRS.15UP.GIN', #same feature that above but with another calculation way
             'IT.NET.USER.P2',
             'SP.POP.TOTL',
             'SP.POP.1524.TO.UN']
mask = df['Indicator Code'].isin(list_feat)
df = df[columns][mask]

# get the last two features from the country dataset
columns = ['Country Code', 'Region', 'Income Group']
data = pd.merge(df, raw_country[columns].dropna(), how='inner', on='Country Code', copy=False)
data.info()

NameError: name 'df' is not defined

On note que l'année 2017 possède aucune valeur non nulle, on ne va donc pas garder cette année. Pour choisir entre 2015 et 2016 regardons quels sont les indicateurs manquants.

In [7]:
data.drop(['2015', '2017'], axis=1).dropna()['Indicator Code'].value_counts()

NameError: name 'data' is not defined

In [ ]:
data.drop(['2016', '2017'], axis=1).dropna()['Indicator Code'].value_counts()

N'ayant aucune donnée de population pour les 15-24 ans en 2016, il est préférable de continuer avec les données de 2015. De plus, on remarque que très peu de données sont disponibles pour le coefficient de Gini. On va donc devoir faire sans.

In [8]:
list_feat = ['NY.GDP.PCAP.PP.CD',
             'IT.NET.USER.P2',
             'SP.POP.TOTL',
             'SP.POP.1524.TO.UN']
mask = data['Indicator Code'].isin(list_feat)
data = data[mask].drop(['2016', '2017'], axis=1)

NameError: name 'data' is not defined

Pour finir, nous allons changer l'organisation de notre table afin d'avoir les indicators comme variable (en colonne)

In [9]:
df = pd.pivot_table(data, values='2015', index='Country Code', columns='Indicator Name')
data = (pd.merge(data.drop(['Indicator Name', 'Indicator Code', '2015'], axis=1).drop_duplicates(),
                 df, on='Country Code', copy=False)
          .set_index('Country Code'))

NameError: name 'data' is not defined

## Explorer les données

### Examiner la stucture des données

Commençons par utiliser la méthode `info()` pour obtenir une description des données sélectionnées.

In [10]:
data.info()

NameError: name 'data' is not defined

On constate que notre jeu de données (après selection de variable) comporte **210 pays et 7 variables**.

La première variable est le nom de chaque pays, nous l'avons gardée d'avoir une description des codes pays. Elle n'a en revanche aucune utilité pour le classement des pays. Les deux autre variables *Region* et *Income Group* sont des variables qualitatives, elles ne peuvent prendre qu'un nombre fini de valeurs différentes qu'on appelle **modalités**. Pour obtenir la liste de ces modalités ainsi que leur effectif, on peut utiliser la méthode `value_counts()`

In [11]:
print("Modalités de la variable 'Region' : \n {}".format(data["Region"].value_counts()))

NameError: name 'data' is not defined

In [ ]:
print("Modalités de la variable 'Income Group' : \n {}".format(data["Income Group"].value_counts()))

Les quatre dernières variables sont des variables quantitatives, la méthode `describe()` permet d'avoir un descriptif des variables numériques.

In [12]:
data.describe()

NameError: name 'data' is not defined

La première ligne compte le nombre de valeurs renseignées, ainsi on peut connaître le nombre de données manquantes pour chaque variable :
- 26 pour le GDP
- 9 pour *internet users*
- 29 pour la population des 15-24 ans
- 1 pour la population totale

Nous verrons par la suite comment traiter ces variable manquantes

De plus, on constate que les variables possèdent des dipersion (attribut std) complètement différentes &ndash; allant jusqu'à 7 ordres de grandeur entre la plus dispersée et la moins dispersée. Il faudra donc les normalise afin d'éviter qu'une des variables écrase les autres.

Une manière rapide de visualiser ces dispersion est de tracer les histogrammes.

In [13]:
data.hist(bins=30, figsize=(8, 8), alpha=0.7)
plt.savefig("charts/hist.png", bbox_inches = "tight")
plt.show()

NameError: name 'data' is not defined

On peut faire plusieurs remarques :
- Ces variables ont des échelles très différentes (ce qu'on avait déja constaté avec la méthode `describe()`). Ce qui confirme la nécessité de normaliser les données.
- La majorité des histogrammes sont fortement dissymétriques, nous essaierons par la suite de transformer ces données afin d'obtenir une distribution plus proche d'une gaussienne.
- La distribution de la variable *internet users* est bimodale.

### Visualisations des données

Étant donné le nombre restreint de données, il serait pratique, pour commencer notre exploration des données, par toutes les tracer sur un même graphique. Cela nous permettrait :
- d'avoir une bonne vision d'ensemble (et voir si certains pays se détachent)
- voir si certaines corrélations existent entre les variables

Au vu des données, le graphique le plus adapté semble être le graphique à bulle. On pourrait tracer le nombre d'utilisateurs internet en fonction du nombre de jeune entre 15-24 ans. On utilisera des cercles pour représenter les points, dont le diamètre seront proportionnels à la population et la couleur fonction des valeurs de PIB.  
La variable **internet users** représente pour 100 personnes le nombre ayant accès à internet, il s'agit donc grosso modo d'un pourcentage. Il serait pertinent d'avoir un format similaire pour la population des 15-24 ans. Pour cela on peut calculer le ratio de de la population des 15-24 sur la population totale, puis multiplier le ratio par 100.
$$\text{15-24, ratio} = \frac{\text{Population, 15-24}}{\text{Population, total}} \times 100$$ 

In [14]:
# add 15-24 population ratio 
data['Population, ages 15-24, ratio'] = data.iloc[:, -2]/data.iloc[:, -1] * 100

NameError: name 'data' is not defined

In [ ]:
data.head()

In [ ]:
plot = data.plot(kind='scatter', x=-1, y=-4, alpha=0.7,
                 s=data.iloc[:, -2]/1000000, figsize=(8, 6),
                 c=3, cmap=plt.get_cmap('jet'), colorbar=True)
# Personnalize the plot
title = "Population ayant accès à internet dans le monde"
subtitle = "Personne ayant accès à internet en 2015 vs la population des 15-24 ans."
file = "bubble_plot"
vs.nice_plot(plot, legend=False, title=title, subtitle=subtitle, file=file)

Les bulles semblent être alignées sur une droite, ce qui indique l'existence d'une corrélation linéaire entre le nombre d'utilisateurs d'internet et la population des 15-24 ans. On remarque aussi que plus la plus population d'un pays est jeune plus le PIB/hab est faible, il y aurait donc la aussi une corrélation entre les variables. En revanche, il ne semble y avoir aucune corrélation entre la taille des bulles (ie. la population totale) et les autres variables.

### Recherche de corrélations entre les variables quantitatives

Un moyen facile de vérifier les corrélations entre les couples de variables est de calculer la matrice des corrélations.

In [15]:
data.corr()

NameError: name 'data' is not defined

Cette matrice nous donne les coefficients de corrélations (appelé également r de Pearson) entre les variables. Ces coefficients sont compris entre -1 (totalement anti-corrélé) et 1 (totalement corrélé). Un coefficient proche de 0 signifie l'absence de corrélation **linéaire**. Ainsi un r proche de 0 ne signifie pas nécessairement l'absence de corrélation. C'est pourquoi, on aime en plus de la matrice des corrélations tracer chaque couple de variable, pour celà on utilise la méthode `scatter_matrix()`.

In [16]:
attributes = data.columns.to_list()
attributes = attributes[3: 5] + attributes [6:]
pd.plotting.scatter_matrix(data[attributes], figsize=(10, 10))
plt.savefig("charts/pairplots.png", bbox_inches = "tight")
plt.show()

NameError: name 'data' is not defined

On constate que la population est bien complètement décorélée des autres variables. En revanche on peut noter que le PIB semble plutôt avoir une relation exponentielle avec les utilisateurs internet et la population des 15-24 ans. Prenons son logarithme et essayons de voir si on obtient une meilleure corrélation.

In [17]:
data["GDP per capita, PPP (log)"] = np.log(data.iloc[:, 3].values)
attributes = data.columns.to_list()
attributes = attributes[4: 5] + attributes [7:]
pd.plotting.scatter_matrix(data[attributes], figsize=(9, 9))
plt.savefig("charts/pairplots_log.png", bbox_inches = "tight")
plt.show()
data[attributes].corr()

NameError: name 'data' is not defined

On constate que le passage au logarithme a permis de linéariser la corrélation avec les autres variables. On obtient ainsi une augmentation significative des coefficients de Pearson.

Nous en avons fini avec la phase d'exploration des données, nous allons pouvoir commencer la préparation de ces dernières avec notamment le traitement des valeurs manquantes.

### Recherche des corrélations avec les variables qualitatives

Pour finir notre exploration de données, nous allons vérifier entre nos variable numériques et les deux variables catégorielles que sont *Region* et *Income Group*. Pour celà nous allons utiliser une représentation en **boîte à moustache**, qui permet de voir la distribution des différentes variables pour chaque catégorie.

In [18]:
attribs = data.columns.to_list()
num_features = attribs[4: 5] + attribs [7:]
by = "Income Group"
file = "{}_boxplot".format(by)
vs.my_box_plot(data, num_features, by, file)

NameError: name 'data' is not defined

On observe une bonne corrélation entre les classes de revenue et les variables quantitatives. Etant directement liée à la variable PIB on s'y attendait.

In [19]:
by = "Region"
file = "{}_boxplot".format(by)
vs.my_box_plot(data, num_features, by, file)

NameError: name 'data' is not defined

Il n'est pas évident que la région puisse permettre d'expliquer certaines des variables. la distribution très resserré pour l'Amérique du Nord est uniquement liée au fait que le nombre d'individus présents dans cette région soit très faible (2). Une **Analyse de la Variance (ANOVA)** a été réalisée pour confirmer ou infirmer l'absence de corrélation.

L'hypothèse $\mathcal{H_{0}}$ qu'il n'y a pas de corrélation a été testée.

Les degrés de libertés et la Somme des Carrés des différents variations ont d'abord été calculés :
- **variation totale** :
    - degrés de liberté : $n - 1$ (avec $n$ le nombre d'individus)
    - SCT : $\sum\limits_{i=1}^{p}\sum\limits_{j=1}^{n_{i}} (y_{ij}-\bar{y})^{2}$
- **variation interclasse** :
    - degrés de liberté : $p - 1$ (avec $p$ le nombre de modalités)
    - SCM : $\sum\limits_{i=1}^{p} n_{i}\times(\bar{y_{i}}-\bar{y})^{2}$
- **variation intraclasse** :
    - degrés de liberté : $n - p$
    - SCR : $\sum\limits_{i=1}^{p}\sum\limits_{j=1}^{n_{i}} (y_{ij}-\bar{y_{i}})^{2}$
    
Puis le test statistique a été effectué en calculant :
1. $\eta^{2} = \frac{SCM}{SCT}$
2. $F_{calc} = \dfrac{\frac{SCM}{p-1}}{\frac{SCR}{n-p}} \sim\mathcal{F}(p-1, n-p)$
3. $p_{value}$

Une $p_{value}$ inférieur à 1% signifie qu'on peut rejeter $\mathcal{H_{0}}$, i.e. conclure qu'il y a corrélation.

In [20]:
attribs = data.columns.to_list()
num_features = attribs[4: 5] + attribs [7:]
by = "Region"
st.anova_Ftest(data, num_features, by)

NameError: name 'data' is not defined

L'ANOVA indique l'existence d'une corrélation entre les variables numériques et la région.

## Préparer les données

### Nettoyer les données

Comme nous l'avons vu précédemment, plusieurs variables comportent des valeurs manquantes. Pour les corriger, nous avons trois solutions :
1. supprimer les pays
2. supprimer les variables
3. les remplacer par une valeur

Commençons par voir pour chaque variable combien de données sont manquantes et ce que ça représente en %.

In [21]:
df_nan = st.na_count(data)
df_nan

NameError: name 'st' is not defined

Pour chaque variable le nombre de valeurs manquantes reste raisonnable, il serait donc dommage de les supprimer. De plus nous avons vu qu'en dehors de la population totale, il existait de forte corrélations. Ces corrélations nous permettrons de remplacer ses données manquantes par une valeur approchée. Commençons quels sont les pays qui possédent au moins une donnée manquante pour la *population totale* et les *utilisateurs internet* (les autres données manquantes pourront toujours être calculées à partir de cette dernière).

In [22]:
mask = data.iloc[:, 4].isnull() | data.iloc[:, 6].isnull()
data[mask]

NameError: name 'data' is not defined

Le pays qui n'a pas de données de pupulation est l'Érythrée. Cependant, le nombre d'utilisateurs internet étant renseigné on pourra calculer une valeur approchée du pourcentage des 15-24 ans, valeur qu'on combinera avec la population des 15-24 ans pour trouver la population totale.

En revanche, pour la variable *internet users*, 5 pays n'ont que la variable population totale de renseignée. On ne pourra donc pas calculer les valeurs manquantes et nous serons obliger de les supprimer l'analyse. Comme il s'agit d'îles à très faible population (<200'000) ce n'est pas gênant &ndash; leur population est trop faible pour être considéré comme des cibles potentielles.

In [23]:
index_to_del = ["ASM", "CHI", "IMN", "MNP", "TCA"]
data_cleaned = data.drop(index_to_del)

NameError: name 'data' is not defined

### Imputer les données manquantes

Pour imputer les données nous allons utiliser un algorithme très classique le **kNN (k-Nearest Neighbours)**. Cet algorithme va regarder les k plus proches pays de celui auquel il manque une donnée. Puis va calculer la moyenne de ces pays pour la variable manquante et assigner cette moyenne à la donnée manquante. Cet algorithme donne de bons résultats pour l'imputation.

Nous allons cependant devoir un peu retravailler nos données avant de l'utiliser. Nous avons vu que les variables *Region* et *Income Group* étaient corrélées aux variables que nous voulons imputer. Nous aimerions alors les utiliser pour chercher les plus proches voisins. Cependant, il s'agit de variables qualitatives or les algorithmes préfèrent les nombres nous allons donc devoir la convertir en nombre. Pour se faire, nous pourrions utiliser la méthode `OrdinalEncoder()` de ScikitLearn. Cette encodeur va assigner un nombre entier à chaque catégorie. Le problème avec cette représentation, c'est que l'algorithme va considérer que deux valeurs voisines sont plus semblables que deux valeurs éloignées. Ce qui n'a pas vraiment de sens pour des catégories. On peut corriger ce problème en appelant directement `OneHotEncoder()` (qui peut s'appliquer directement sur des chaînes de caractères depuis la version 0.20 de Scikit-Learn.).

Nous avons aussi vu que nos données avaient des échelles très différentes, nous allons donc devoir les normaliser. Ce qu'on peut faire facilement avec la méthode `StandardScaler()` de Scikit-Learn.

Nous pouvons appliquer ces deux transformateurs (`OneHotEncoder()` pour les variables qualitatives et `StandardScaler()` pour les variables quantitatives) en parallèle grâce à la classe `ColumnTransformer`

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

num_attribs = list(num_features)
cat_attribs = data.columns.to_list()[1: 3]
full_pipeline = ColumnTransformer([('num', StandardScaler(), num_attribs),
                                   ('cat', OneHotEncoder(), cat_attribs)])
data_prepared = full_pipeline.fit_transform(data_cleaned)

print(data_prepared.shape)
print(np.isnan(data_prepared).sum())
data_prepared

NameError: name 'num_features' is not defined

On peut maintenant utiliser notre kNN à l'aide de la classe (expérimentale) `IterativeImputer` de Scikit-Learn. Cette classe permet de réaliser des imputations multivariées (utilisent toutes les autres variables pour imputer la donnée manquante) de manière itérative (round-robin fashion).

In [ ]:
full_pipeline.transformers_[0][1]

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
estimator = KNeighborsRegressor(n_neighbors=3)
imputer = IterativeImputer(estimator)
data_tr = imputer.fit_transform(data_prepared)
np.isnan(data_tr).sum()

Il nous reste à rajouter les données de population (totale et 15-24 ans) pour déterminer les dernières données manquantes.

In [ ]:
std_scaler = full_pipeline.transformers_[0][1]
data_f = (pd.DataFrame(std_scaler.inverse_transform(data_tr[:, :3]),
                           columns=num_attribs, index=data_cleaned.index)
                .merge(data_cleaned.iloc[:, 5:7], left_index=True, right_index=True)
                .merge(data_cleaned.iloc[:, 0], left_index=True, right_index=True)
                .set_index('Country Name'))                
data_f.head()

On peut maintenant imputer la donnée manquante pour la variable population totale ainsi que celles pour la population des 12-24 ans.

In [123]:
# impute population, total
idx = data_f.iloc[:, -1].isnull().to_numpy().nonzero()[0]
data_f.iloc[idx, -1] = data_f.iloc[idx, -2] / data_f.iloc[idx, -4] * 100

# impute population 15-24,total
idx = data_f.iloc[:, -2].isnull().to_numpy().nonzero()[0]
data_f.iloc[idx, -2] = data_f.iloc[idx, -1] * data_f.iloc[idx, -4] / 100

## Création d'un modèle permettant de trier les pays en fonction de leur pertinance

### Création d'un score

Maintenant que nous avons des données complètes, nous pouvons calculer des scores. Pour commencer nous allons discrétiser nos données de manière à avoir une note pour chaque variable ces notes seront entre 0 (exclu) et 5 (inclu). Ainsi il sera facile de visualiser les pays via des graphiques radars et pour les classer, nous aurons qu'à calculer la note moyenne sur les cinq variables.  
A ce propos, j'ai préféré garder la population totale des 15-24 et non seulement le ratio pour donner plus importance à la population qui est la variable la plus importante vu qu'elle détermine la taille potentiel du marché.

In [126]:
# Discretize all columns based on decile
cols = data_f.columns.to_list()
for i, c in enumerate(cols):
    if i == O:
        serie = pd.qcut(data_f[c], 10, labels=range(1, 11)/2)
        data_f_cat = pd.DataFrame({c: serie}, index=data_f.index)
    else:
        data_f_cat[c] = pd.qcut(data_f[c], 10, labels=range(1, 11)/2)

# Add Score and Rank column
data_f_cat['Score'] = data_f_cat.mean(axis=1)
data_f_cat['Rank'] = data_f_cat['Score'].rank(method='min', ascending=False)

### Visualisation des 5 pays les plus pertinants

In [127]:
mask = data_final['Rank'] < 6
top_countries = data_final.sort_values('Rank')[mask].index.tolist()
print(top_countries)

['China', 'India', 'United States', 'Brunei Darussalam', 'Luxembourg']


C:\Users\pe.ragettli\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi

categories = list(df)[1:]
values = df.mean().values.flatten().tolist()
values += values[:1] # repeat the first value to close the circular graph
angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
angles += angles[:1]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8),
                       subplot_kw=dict(polar=True))

plt.xticks(angles[:-1], categories, color='grey', size=12)
plt.yticks(np.arange(1, 6), ['1', '2', '3', '4', '5'],
           color='grey', size=12)
plt.ylim(0, 5)
ax.set_rlabel_position(30)
 
ax.plot(angles, values, linewidth=1, linestyle='solid')
ax.fill(angles, values, 'skyblue', alpha=0.4)

plt.show()